In [1]:
import PyPDF2

def extract_text_from_pdf_pypdf2(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

pdf_text = extract_text_from_pdf_pypdf2("The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf")

print(pdf_text[:1000])

TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITIONTheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION
JACQUELINE L. LONGE, EDITOR
DEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR
VOLUME
C-F2STAFF
Jacqueline L. Longe, Project Editor
Deirdre S. Blanchfield, Associate Editor
Christine B. Jeryan, Managing Editor
Donna Olendorf, Senior Editor
Stacey Blachford, Associate Editor
Kate Kretschmann, Melissa C. McDade, Ryan
Thomason, Assistant Editors
Mark Springer, Technical Specialist
Andrea Lopeman, Programmer/Analyst
Barbara J. Yarrow, Manager, Imaging and Multimedia
Content
Robyn V . Young, Project Manager, Imaging and
Multimedia Content
Dean Dauphinais, Senior Editor, Imaging and
Multimedia Content
Kelly A. Quin, Editor, Imaging and Multimedia Content
Leitha Etheridge-Sims, Mary K. Grimes, Dave Oblender,
Image Catalogers
Pamela A. Reed, Imaging Coordinator
Randy Bassett, Imaging Supervisor
Robert Duncan, Senior Imaging Specialist
Dan Newell, Imaging Specialist
Christine O’Bryan, Graphic Specialist
Maria Fra

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = splitter.split_text(pdf_text)
print(f"Total chunks created: {len(chunks)}")


Total chunks created: 6785


In [3]:
from langchain.embeddings import OpenAIEmbeddings
import os

os.environ["OPENAI_API_KEY"] = "enter your key here"

embed_model = OpenAIEmbeddings(model="text-embedding-3-small")


C:\Users\ASUS\AppData\Local\Temp\ipykernel_21360\187282111.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embed_model = OpenAIEmbeddings(model="text-embedding-3-small")


In [4]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_texts(chunks, embedding=embed_model)
vectorstore.save_local("gale_medical_index")

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name="gpt-3.5-turbo")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21360\2064865790.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo")


In [6]:
def ask_doctor_ai(query):
    result = qa_chain(query)
    print("🤖 Doctor AI:\n", result["result"])
    print("\n📚 Source Snippet:\n", result["source_documents"][0].page_content[:300])

In [7]:
ask_doctor_ai("What are the symptoms of asthma?")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21360\3563023111.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  result = qa_chain(query)


🤖 Doctor AI:
 The symptoms of asthma can include coughing, wheezing, shortness of breath, chest tightness, and difficulty breathing.

📚 Source Snippet:
 inhaled,causes the airways to narrow and pro-
duces symptoms of asthma.
Antibody —A protein, also called immunoglobu-
lin, produced by immune system cells to remove
antigens (the foreign substances that trigger the
immune response).
Fibromyalgia —A condition of debilitating pain,
among other symptom
